In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# --- Load and prepare data ---
X = np.load(f'all_segments.npy').astype(np.float32)
y = np.load(f'all_labels.npy').astype(np.int32)

# Optional: Normalize signals
# X = (X - np.mean(X)) / np.std(X)

# --- Train/Validation Split ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# --- CNN Model for Binary Classification ---
model = models.Sequential([
    layers.Input(shape=(2000, 2)),

    layers.Conv1D(16, kernel_size=7, activation='relu'),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(32, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=2),

    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling1D(),

    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),  # helps prevent overfitting

    layers.Dense(1, activation='sigmoid')  # binary output
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 1994, 16)       │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 997, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 993, 32)        │         2,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 496, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 494, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,153 (43.57 KB)

 Trainable params: 11,153 (43.57 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:

# --- Train Model ---
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_data=(X_val, y_val))


Epoch 1/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7094 - loss: 0.6312 - val_accuracy: 0.7181 - val_loss: 0.5648
Epoch 2/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7190 - loss: 0.5642 - val_accuracy: 0.7199 - val_loss: 0.5359
Epoch 3/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7426 - loss: 0.5350 - val_accuracy: 0.7520 - val_loss: 0.5215
Epoch 4/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7635 - loss: 0.5085 - val_accuracy: 0.7719 - val_loss: 0.4869
Epoch 5/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7801 - loss: 0.4852 - val_accuracy: 0.7971 - val_loss: 0.4531
Epoch 6/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7984 - loss: 0.4584 - val_accuracy: 0.8049 - val_loss: 0.4385
Epoch 7/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.8004 - loss: 0.4282 - val_accuracy: 0.8257 - val_loss: 0.4371
Epoch 8/100
145/145 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8257 - loss: 0.4008 - 

In [ ]:

# --- Save the Model and Label Map ---
model.save(f'models/ecg_cnn_{record_id}.h5')
np.save(f'models/{record_id}_label_classes.npy', label_encoder.classes_)
